In [ ]:
from pypot.sensor import OpenCVCamera
import matplotlib.pyplot as plt

In [ ]:
cam = OpenCVCamera('camera', 0, 20)

In [ ]:
red = cam.filter_red_objects()
red = red[:,:,::-1]
plt.imshow(red)
plt.show()

In [ ]:
yellow = cam.filter_yellow_objects()
yellow = yellow[:,:,::-1]
plt.imshow(yellow)
plt.show()

In [ ]:
green = cam.filter_green_objects()
green = green[:,:,::-1]
plt.imshow(green)
plt.show()

In [ ]:
a,b = ([165,50,50], [182,255,255]) #HSV range for red color
custom = cam.filter_objects_by_color_range(a, b)
im_objects, objects = cam.get_image_objects(custom,approximate=False,draw_contours=True, gaussian_filter_size=9, threshold_limits=[20,255])
im_objects_rgb = im_objects[:,:,::-1]
plt.imshow(im_objects_rgb)
plt.show()
for (contours, center, area) in objects:
    print center, area

In [ ]:
import cv2
def custom_filter(bgr_image):
    """The custom filter must receive an BGR image as a param
    :param image: BGR array image
    """
    image = bgr_image.copy()
    gaussian_filter_size = 9
    threshold_limits = [10,255]
    approximate = True
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray_image, (gaussian_filter_size, gaussian_filter_size), 0)
    thresh = cv2.threshold(blurred, threshold_limits[0], threshold_limits[1], cv2.THRESH_BINARY)[1]
    contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[1]
    objects = []
    for contour in contours:
        if approximate:
            epsilon = 0.1 * cv2.arcLength(contour, True)
            contour = cv2.approxPolyDP(contour, epsilon, True)
        area = cv2.contourArea(contour)
        if area == 0:
            continue
        # compute the center of the contour
        M = cv2.moments(contour)
        try:
            center_x = int(M["m10"] / M["m00"])
            center_y = int(M["m01"] / M["m00"])
            if draw_contours:
                cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
                cv2.circle(image, (center_x, center_y), 4, (255, 255, 255), -1)
            objects.append((contour, (center_x, center_y), area))
        except:
            pass
    return image, objects

In [ ]:
cam.add_custom_filter("custom",custom_filter)

In [ ]:
cam.run_custom_filter("custom")

In [ ]:
cam.close()